In [9]:
import numpy as np
from NonlinearController.mpc_utils import *
# from NonlinearController.controllers import *
from NonlinearController.model_utils import *
from NonlinearController.lpv_embedding import *
import deepSI

In [ ]:
model = deepSI.load_system("NonlinearController/trained_models/unbalanced/ObserverUnbalancedDisk_dt01_nab_4_SNR_30_e250")

get_A, get_B, get_C = velocity_lpv_embedding(model)

nx = model.nx
n_states = nx
x = MX.sym("x",nx,1)
nu = model.nu if model.nu is not None else 1
n_controls = nu
u = MX.sym("u",nu,1)
ny = model.ny if model.ny is not None else 1

A = np.zeros((nx,nx))
B = np.zeros((nx,nu))
C = np.zeros((ny,nx))

x0 = np.zeros((nx,1))
dx = np.zeros((nx,1))
u0 = np.zeros((nu,1))
du = np.zeros((nu,1))

steps = 100
log_x = np.zeros((nx,steps))

for i in range(steps):

    A[:,:] = get_A(x0,dx,u0,du)
    B[:,:] = get_B(x0,dx,u0,du)
    C[:,:] = get_C(x0,dx,u0,du)

    x1 = A @ x0 + B @ u0

    dx = x1-x0
    x0 = x1

    u1 = np.random.normal(size=(nu,1))
    du = u1 - u0
    u0 = u1

    log_x[:,i] = x1[:,0]

plt.plot(log_x[0,:])
plt.plot(log_x[1,:])
plt.show()